<a href="https://colab.research.google.com/github/b1becker/love_to_hate_sentiment_analysis/blob/main/daniil_medvedev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import seaborn as sns
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import warnings
warnings.filterwarnings('ignore')

class MedvedevSentimentVisualizer:
    def __init__(self):
        # Set style for professional look
        plt.style.use('default')
        sns.set_palette("husl")

    def create_sentiment_dashboard(self, avg_negativity, max_negativity, image_path='medvedev.jpeg',
                                 total_posts=5, min_negativity=0.043):
        """Create a comprehensive sentiment dashboard with Medvedev's photo"""

        # Create figure with subplots
        fig = plt.figure(figsize=(16, 10))
        gs = fig.add_gridspec(3, 3, height_ratios=[1, 1.5, 1], width_ratios=[1, 1.5, 1])

        # Main title
        fig.suptitle('DANIIL MEDVEDEV SENTIMENT ANALYSIS DASHBOARD',
                    fontsize=24, fontweight='bold', y=0.95)

        # Try to load and display Medvedev's image
        ax_image = fig.add_subplot(gs[0, 1])
        try:
            img = Image.open(image_path)
            ax_image.imshow(img)
            ax_image.set_title('Daniil Medvedev', fontsize=16, fontweight='bold')
        except FileNotFoundError:
            # Create placeholder if image not found
            ax_image.text(0.5, 0.5, 'MEDVEDEV\nPHOTO\nNOT FOUND',
                         ha='center', va='center', fontsize=14,
                         bbox=dict(boxstyle="round,pad=0.3", facecolor='lightgray'))
            ax_image.set_title('Photo Placeholder', fontsize=16)

        ax_image.axis('off')

        # Sentiment gauge
        ax_gauge = fig.add_subplot(gs[1, 0])
        self.create_sentiment_gauge(ax_gauge, avg_negativity)

        # Main statistics display
        ax_stats = fig.add_subplot(gs[1, 1])
        self.create_stats_display(ax_stats, avg_negativity, max_negativity,
                                min_negativity, total_posts)

        # Sentiment distribution
        ax_dist = fig.add_subplot(gs[1, 2])
        self.create_sentiment_distribution(ax_dist, avg_negativity, max_negativity, min_negativity)

        # Bottom row - additional metrics
        ax_comparison = fig.add_subplot(gs[2, :])
        self.create_comparison_bar(ax_comparison, avg_negativity, max_negativity)

        plt.tight_layout()
        plt.subplots_adjust(top=0.90)
        return fig

    def create_sentiment_gauge(self, ax, avg_negativity):
        """Create a circular gauge showing average sentiment"""
        # Create semicircle gauge
        theta = np.linspace(0, np.pi, 100)

        # Background semicircle
        x_bg = np.cos(theta)
        y_bg = np.sin(theta)
        ax.fill(x_bg, y_bg, color='lightgray', alpha=0.3)

        # Color zones
        colors = ['green', 'yellow', 'orange', 'red']
        zones = [0.125, 0.25, 0.375, 0.5]

        for i, (color, zone) in enumerate(zip(colors, zones)):
            theta_zone = np.linspace(i*np.pi/4, (i+1)*np.pi/4, 20)
            x_zone = np.cos(theta_zone)
            y_zone = np.sin(theta_zone)
            ax.fill_between(x_zone, 0, y_zone, color=color, alpha=0.6)

        # Needle pointing to average negativity
        needle_angle = np.pi * (1 - avg_negativity * 2)  # Scale to semicircle
        needle_x = [0, 0.8 * np.cos(needle_angle)]
        needle_y = [0, 0.8 * np.sin(needle_angle)]
        ax.plot(needle_x, needle_y, 'k-', linewidth=4)
        ax.plot(0, 0, 'ko', markersize=8)

        # Labels
        ax.text(0, -0.3, f'{avg_negativity:.3f}', ha='center', va='center',
               fontsize=16, fontweight='bold')
        ax.text(0, -0.45, 'Average Negativity', ha='center', va='center', fontsize=12)

        ax.set_xlim(-1.2, 1.2)
        ax.set_ylim(-0.6, 1.2)
        ax.set_aspect('equal')
        ax.axis('off')
        ax.set_title('Sentiment Gauge', fontsize=14, fontweight='bold')

    def create_stats_display(self, ax, avg_negativity, max_negativity,
                           min_negativity, total_posts):
        """Create a statistics display panel"""
        ax.axis('off')

        # Background
        rect = patches.Rectangle((0.05, 0.05), 0.9, 0.9, linewidth=2,
                               edgecolor='black', facecolor='lightblue', alpha=0.1)
        ax.add_patch(rect)

        # Statistics text
        stats_text = f"""
KEY METRICS

📊 Total Posts Analyzed: {total_posts}

📈 Average Negativity: {avg_negativity:.3f}

🔴 Most Negative Score: {max_negativity:.3f}

🟢 Least Negative Score: {min_negativity:.3f}

📊 Negativity Range: {max_negativity - min_negativity:.3f}

⚡ Sentiment Volatility: {'HIGH' if max_negativity - min_negativity > 0.3 else 'MODERATE' if max_negativity - min_negativity > 0.15 else 'LOW'}
        """

        ax.text(0.1, 0.5, stats_text, fontsize=12, va='center',
               bbox=dict(boxstyle="round,pad=0.5", facecolor='white', alpha=0.8))

        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)

    def create_sentiment_distribution(self, ax, avg_negativity, max_negativity, min_negativity):
        """Create a sentiment distribution visualization"""
        # Sample data points (since we only have summary stats)
        sentiment_scores = [min_negativity, avg_negativity*0.8, avg_negativity,
                          avg_negativity*1.2, max_negativity]

        # Create histogram-style bars
        bars = ax.bar(range(len(sentiment_scores)), sentiment_scores,
                     color=['green', 'yellow', 'orange', 'red', 'darkred'],
                     alpha=0.7)

        # Add value labels on bars
        for i, (bar, score) in enumerate(zip(bars, sentiment_scores)):
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                   f'{score:.3f}', ha='center', va='bottom', fontweight='bold')

        ax.set_xlabel('Post Samples', fontsize=12)
        ax.set_ylabel('Negativity Score', fontsize=12)
        ax.set_title('Sentiment Distribution', fontsize=14, fontweight='bold')
        ax.set_xticks(range(len(sentiment_scores)))
        ax.set_xticklabels(['Least\nNegative', 'Below\nAverage', 'Average',
                           'Above\nAverage', 'Most\nNegative'], fontsize=10)
        ax.grid(True, alpha=0.3)

    def create_comparison_bar(self, ax, avg_negativity, max_negativity):
        """Create comparison bars showing different sentiment thresholds"""
        categories = ['Neutral\n(0.0-0.1)', 'Slightly Negative\n(0.1-0.2)',
                     'Moderately Negative\n(0.2-0.3)', 'Very Negative\n(0.3-0.4)',
                     'Extremely Negative\n(0.4+)']

        thresholds = [0.05, 0.15, 0.25, 0.35, 0.45]
        colors = ['lightgreen', 'yellow', 'orange', 'red', 'darkred']

        bars = ax.bar(categories, thresholds, color=colors, alpha=0.6,
                     label='Sentiment Thresholds')

        # Add markers for actual values
        ax.axhline(y=avg_negativity, color='blue', linestyle='--', linewidth=3,
                  label=f'Average Negativity ({avg_negativity:.3f})')
        ax.axhline(y=max_negativity, color='red', linestyle='--', linewidth=3,
                  label=f'Max Negativity ({max_negativity:.3f})')

        # Highlight where Medvedev's scores fall
        for i, threshold in enumerate(thresholds):
            if avg_negativity <= threshold:
                bars[i].set_edgecolor('blue')
                bars[i].set_linewidth(3)
                break

        ax.set_ylabel('Negativity Score', fontsize=12)
        ax.set_title('Medvedev Sentiment vs. Standard Negativity Levels',
                    fontsize=16, fontweight='bold')
        ax.legend(loc='upper left')
        ax.grid(True, alpha=0.3)

        # Add interpretation text
        interpretation = self.interpret_sentiment(avg_negativity, max_negativity)
        ax.text(0.98, 0.95, interpretation, transform=ax.transAxes,
               fontsize=11, va='top', ha='right',
               bbox=dict(boxstyle="round,pad=0.5", facecolor='lightyellow'))

    def interpret_sentiment(self, avg_negativity, max_negativity):
        """Provide interpretation of sentiment scores"""
        if avg_negativity < 0.15:
            overall = "MOSTLY POSITIVE"
        elif avg_negativity < 0.25:
            overall = "MILDLY NEGATIVE"
        elif avg_negativity < 0.35:
            overall = "MODERATELY NEGATIVE"
        else:
            overall = "HIGHLY NEGATIVE"

        if max_negativity > 0.4:
            extreme = "Some EXTREMELY harsh criticism"
        elif max_negativity > 0.3:
            extreme = "Some very critical posts"
        else:
            extreme = "Criticism is relatively mild"

        return f"INTERPRETATION:\n{overall} sentiment overall\n{extreme}"

    def save_dashboard(self, fig, filename='medvedev_sentiment_dashboard.png', dpi=300):
        """Save the dashboard as high-quality image"""
        fig.savefig(filename, dpi=dpi, bbox_inches='tight', facecolor='white')
        print(f"Dashboard saved as {filename}")

    def create_simple_card(self, avg_negativity, max_negativity, image_path='medvedev.jpeg'):
        """Create a simple social media style card"""
        fig, ax = plt.subplots(1, 1, figsize=(10, 8))

        # Background gradient
        gradient = np.linspace(0, 1, 256).reshape(256, -1)
        gradient = np.vstack((gradient, gradient))
        ax.imshow(gradient, aspect='auto', cmap='RdYlBu_r', alpha=0.3)

        # Try to add image
        try:
            img = Image.open(image_path)
            # Resize image to fit
            img_resized = img.resize((200, 200))
            ax.imshow(img_resized, extent=[0.1, 0.4, 0.6, 0.9])
        except FileNotFoundError:
            ax.text(0.25, 0.75, '📸\nMEDVEDEV', ha='center', va='center',
                   fontsize=20, fontweight='bold')

        # Main text
        ax.text(0.6, 0.8, 'DANIIL MEDVEDEV', fontsize=24, fontweight='bold',
               ha='center', color='darkblue')
        ax.text(0.6, 0.7, 'SENTIMENT ANALYSIS', fontsize=16, ha='center',
               style='italic')

        # Metrics
        ax.text(0.6, 0.5, f'Average Negativity: {avg_negativity:.3f}',
               fontsize=18, ha='center', fontweight='bold',
               bbox=dict(boxstyle="round,pad=0.3", facecolor='lightcoral'))

        ax.text(0.6, 0.35, f'Peak Negativity: {max_negativity:.3f}',
               fontsize=18, ha='center', fontweight='bold',
               bbox=dict(boxstyle="round,pad=0.3", facecolor='red', alpha=0.7))

        # Interpretation
        sentiment_emoji = "😤" if avg_negativity > 0.3 else "😕" if avg_negativity > 0.2 else "😐"
        ax.text(0.6, 0.15, f'Public Mood: {sentiment_emoji}',
               fontsize=20, ha='center', fontweight='bold')

        ax.set_xlim(0, 1)
        ax.set_ylim(0, 1)
        ax.axis('off')

        plt.tight_layout()
        return fig

# Example usage
def create_medvedev_graphics():
    """Main function to create Medvedev sentiment graphics"""
    # Your actual data
    avg_negativity = 0.263
    max_negativity = 0.457
    min_negativity = 0.043
    total_posts = 5

    # Initialize visualizer
    viz = MedvedevSentimentVisualizer()

    # Create comprehensive dashboard
    dashboard = viz.create_sentiment_dashboard(
        avg_negativity=avg_negativity,
        max_negativity=max_negativity,
        min_negativity=min_negativity,
        total_posts=total_posts,
        image_path='medvedev.jpeg'
    )

    # Save dashboard
    viz.save_dashboard(dashboard, 'medvedev_sentiment_dashboard.png')

    # Create simple card version
    card = viz.create_simple_card(avg_negativity, max_negativity, 'medvedev.jpeg')
    card.savefig('medvedev_sentiment_card.png', dpi=300, bbox_inches='tight',
                facecolor='white')

    plt.show()

if __name__ == "__main__":
    create_medvedev_graphics()

# Additional requirements:
# pip install pillow  # For image processing